#**Import Libraries**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import cv2
#import glob
import numpy as np
from PIL import Image
#import matplotlib.pyplot as plt
import hashlib
from collections import defaultdict

#**Implementasi Kode**

**Path Folder GDrive**

In [4]:
# Path ke dataset di Google Drive
base_path = "/content/drive/My Drive/FaceRec_NN/Data_Resize30x30"
output_path = "/content/drive/My Drive/FaceRec_NN/Data_Flatten/Flattened_dataset.npz"

**Periksa Jumlah File tiap Folder**

In [5]:
def count_files_in_folders(base_path):
    total_files = 0
    folder_file_count = {}

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):  # Pastikan hanya menghitung folder
            file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            folder_file_count[folder] = file_count
            total_files += file_count

    # Menampilkan hasil
    print("Jumlah file per folder:")
    for folder, count in folder_file_count.items():
        print(f"- {folder}: {count} file")

    print(f"\nTotal file: {total_files} file")
    return total_files

total = count_files_in_folders(base_path)

Jumlah file per folder:
- Satwika: 80 file
- Mikhael: 80 file
- Puti: 80 file
- Raesa: 80 file
- Khansa: 80 file
- Hanna: 80 file
- Hafidz: 80 file
- Haidar: 80 file
- Azmira: 80 file
- Dimas: 80 file
- David: 80 file
- Keiko: 80 file
- Fadhli: 80 file
- Fadlin: 80 file
- Toni: 80 file

Total file: 1200 file


**Verifikasi Data**

In [6]:
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = Image.open(file_path)
                print(f"Image: {file_path}, Mode: {img.mode}, Shape: {img.size}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_10.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_1.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_13.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_19.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_11.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_15.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_14.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_12.jpg, Mode: L, Shape: (30, 30)
Image: /content/drive/My Drive/FaceRec_NN/Data_Resize30x30/Satwika/Satwika_30x30_17.jpg, 

**Program Flatten**

In [12]:
def flatten_images(base_path, output_path=None):
    images, labels = [], []
    label_map = {}  # Menyimpan mapping nama folder ke indeks
    label_idx = 0  # Indeks untuk penamaan label

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if not os.path.isdir(folder_path):
            continue  # Lewati jika bukan folder

        # Menambahkan mapping folder ke label
        label_map[folder] = label_idx
        label_idx += 1

        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = Image.open(file_path).convert('RGB')  # Convert to RGB mode
                img = img.convert('L')  # Convert to Grayscale untuk konsistensi
                flattened = np.array(img).flatten() / 255.0  # Flatten dan normalisasi

                images.append(flattened)  # Menambahkan gambar ke dataset
                labels.append(label_map[folder])  # Menambahkan label berdasarkan nama folder

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    # Konversi ke array numpy
    images = np.array(images)
    labels = np.array(labels)

    # One-hot encoding untuk labels
    num_classes = len(label_map)
    labels_one_hot = np.zeros((labels.size, num_classes))
    labels_one_hot[np.arange(labels.size), labels] = 1

    # Simpan dataset jika output_path disediakan
    if output_path:
        np.savez(output_path, images=images, labels=labels_one_hot)
        print(f"Dataset saved to {output_path}")

    return images, labels_one_hot

**Pemrosesan Flatten Data**

In [13]:
X, y = flatten_images(base_path, output_path = output_path)
print(f"Dataset shape: {X.shape}, Labels shape: {y.shape}")

Dataset saved to /content/drive/My Drive/FaceRec_NN/Data_Flatten/Flattened_dataset.npz
Dataset shape: (1200, 900), Labels shape: (1200, 15)


**Membaca Flatten Data**

In [14]:
# Load dataset flatten
data = np.load("/content/drive/My Drive//FaceRec_NN/Data_Flatten/Flattened_dataset.npz")
images = data['images']
labels = data['labels']

print(f"Data shape: {images.shape}, Labels shape: {labels.shape}")

Data shape: (1200, 900), Labels shape: (1200, 15)
